Read data from file.

In [1]:
import pandas as pd
import numpy as np

In [11]:
cab_data = pd.read_csv("Cab_Data.csv")
city = pd.read_csv("City.csv")
customer_id = pd.read_csv("Customer_ID.csv")
transaction_id = pd.read_csv("Transaction_ID.csv")

Extract `Pink Cab` and `Yellow Cab` data from `cab_data`.

In [18]:
pink = cab_data[cab_data["Company"] == "Pink Cab"]
yellow = cab_data[cab_data["Company"] == "Yellow Cab"]

Average summary data from the two companies.

In [25]:
pink_summary = ["Pink Cab", pink["KM Travelled"].mean(), pink["Price Charged"].mean(), pink["Cost of Trip"].mean()]
yellow_summary = ["Yellow Cab", yellow["KM Travelled"].mean(), yellow["Price Charged"].mean(), yellow["Cost of Trip"].mean()]

In [29]:
summary = pd.DataFrame([pink_summary, yellow_summary], columns=["Company", "KM Travelled", "Price Charged", "Cost of Trip"])
summary

,Company,KM Travelled,Price Charged,Cost of Trip
0,Pink Cab,22.559917,310.800856,248.148682
1,Yellow Cab,22.569517,458.181990,297.922004


From the table above, we have:
> KM Travelled: Almost the same.
> Price Charged: Yellow Cab charges higher.
> Cost of Trip: Yellow ab has a higher cost.

Then, we calculate the numbers of orders and total profits for the two companies.

In [36]:
pink_orders = len(pink)
yellow_orders = len(yellow)

print(f"Pink Cab has {pink_orders} orders.\nYellow Cab has {yellow_orders} orders.")

Pink Cab has 84711 orders.
Yellow Cab has 274681 orders.


In [37]:
pink_profits = (pink["Price Charged"] - pink["Cost of Trip"]).mean()
yellow_profits = (yellow["Price Charged"] - yellow["Cost of Trip"]).mean()

print(f"Pink Cab's profits: {pink_profits}.\nYellow Cab's profits: {yellow_profits}.")

Pink Cab's profits: 62.65217410961977.
Yellow Cab's profits: 160.25998584103016.
